In [1]:
import numpy as np
import json

In [2]:
with open('arab-boy.json') as fopen:
    arab_boy = json.load(fopen)
arab_boy = [i[1] for i in arab_boy]
arab_boy = list(set(' '.join(arab_boy).split()))

In [3]:
with open('arab-girl.json') as fopen:
    arab_girl = json.load(fopen)
arab_girl = [i[1] for i in arab_girl]
arab_girl = list(set(' '.join(arab_girl).split()))

In [4]:
with open('chinese-boy.json') as fopen:
    chinese_boy = json.load(fopen)
chinese_boy = [i[1] for i in chinese_boy]
chinese_boy = list(set(' '.join(chinese_boy).split()))

In [5]:
with open('chinese-girl.json') as fopen:
    chinese_girl = json.load(fopen)
chinese_girl = [i[1] for i in chinese_girl]
chinese_girl = list(set(' '.join(chinese_girl).split()))

In [13]:
with open('indian-boy.json') as fopen:
    indian_boy = json.load(fopen) 
indian_boy = list(set(indian_boy))

In [14]:
with open('indian-girl.json') as fopen:
    indian_girl = json.load(fopen) 
indian_girl = list(set(indian_girl))

In [15]:
chinese = list(set(chinese_boy + chinese_girl))
arabic = list(set(arab_boy + arab_girl))
indian = list(set(indian_boy + indian_girl))

In [10]:
# !wget https://raw.githubusercontent.com/rossgoodwin/american-names/master/surnames.json

In [16]:
with open('surnames.json') as fopen:
    american = json.load(fopen)

In [18]:
mapping = {0: arabic, 1: chinese, 2: indian, 3: american}
mixed_mapping = {0: [american, chinese], 1: [indian, arabic], 2: [chinese, arabic]}

In [24]:
import random

def generate_name(length):
    r = random.randint(0,4)
    if r == 4:
        r = mixed_mapping[random.randint(0, 2)]
        l, r = random.choice(r[0]), random.choice(r[1])
        name = f'{l} {r}'
    else:
        s = mapping[r]
        name = ' '.join(random.sample(s, length))
        while length == 1 and len(name) < 5:
            name = ' '.join(random.sample(arabic, length))
    return name

In [37]:
with open('entities-data-v4.json') as fopen:
    entities = json.load(fopen)

In [38]:
results = []
i = 0
while i < len(entities['label']):
    r = []
    if entities['label'][i] == 'person':
        while entities['label'][i] == 'person':
            r.append(i)
            i += 1
        print(r)
        results.append(r)
    i += 1

[7, 8]
[16, 17]
[32]
[46]
[54]
[74]
[82]
[85]
[119, 120]
[122, 123]
[125]
[127]
[148]
[165]
[169]
[171]
[173]
[187]
[190]
[208]
[231]
[236, 237]
[240, 241]
[255]
[280, 281]
[317, 318]
[338, 339, 340, 341]
[371]
[416]
[472, 473]
[478, 479]
[483, 484, 485]
[489, 490]
[494, 495, 496]
[500, 501]
[505, 506, 507]
[509]
[513, 514]
[516]
[520, 521]
[526, 527, 528, 529]
[538, 539, 540, 541, 542, 543]
[591, 592]
[621, 622, 623, 624]
[632, 633, 634, 635, 636]
[641, 642, 643, 644, 645]
[652, 653, 654, 655, 656, 657, 658, 659]
[663]
[712]
[714]
[760]
[762, 763, 764]
[791]
[802, 803]
[826]
[954, 955]
[1049, 1050]
[1056]
[1065]
[1076]
[1086]
[1091, 1092]
[1098]
[1114]
[1124]
[1133]
[1165]
[1205, 1206]
[1209, 1210]
[1217, 1218]
[1229, 1230]
[1236]
[1242]
[1253]
[1262]
[1268, 1269]
[1280]
[1282]
[1285]
[1298]
[1320]
[1331]
[1371, 1372]
[1378]
[1390]
[1406, 1407]
[1414, 1415]
[1417, 1418]
[1424, 1425]
[1463]
[1553, 1554, 1555]
[1557, 1558]
[1561]
[1569, 1570, 1571]
[1573, 1574]
[1588]
[1611]
[1617]
[164

In [41]:
from sklearn.model_selection import train_test_split

train_results, test_results = train_test_split(results, test_size = 0.2)

In [42]:
import math

def generate_index(l, name, texts, labels, length):
    cp, indices = [], []
    b = length - len(l)
    left = math.ceil(b / 2)
    right = b - left
    minus = l[0] - left
    if minus < 0:
        absolute = np.abs(minus)
        right += absolute
        left -= absolute

    for i in range(l[0] - left, l[0]):
        cp.append(texts[i])
        indices.append(labels[i])

    cp.extend(name)
    indices.extend([labels[l[0]] for _ in range(len(name))])
    try:
        for i in range(l[-1] + 1, l[-1] + right + 1):
            cp.append(texts[i])
            indices.append(labels[i])
    except Exception as e:
        print(e)
        pass
    
    return cp, indices

In [46]:
train_X, train_Y = [], []

for r in train_results:
    for _ in range(10):
        n = generate_name(len(r)).split()
        x, y = generate_index(r, n, entities['text'], entities['label'], 20)
        if len(x) != len(y):
            print('len not same')
            continue
        train_X.append(x)
        train_Y.append(y)
        
len(train_X)

17860

In [47]:
test_X, test_Y = [], []

for r in test_results:
    for _ in range(10):
        n = generate_name(len(r)).split()
        x, y = generate_index(r, n, entities['text'], entities['label'], 20)
        if len(x) != len(y):
            print('len not same')
            continue
        test_X.append(x)
        test_Y.append(y)
        
len(test_X)

4470

In [48]:
with open('name-augmentation.json', 'w') as fopen:
    json.dump({'train_X': train_X, 'train_Y': train_Y, 'test_X': test_X, 'test_Y': test_Y}, fopen)